**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.1/562.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 5.9 MB/s eta 0:00:00


First let's read the RDF file

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")

<Graph identifier=Nfd411148aea84bcfaab703c522b1d6fa (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "LivingThing" with RDFLib and SPARQL**

In [4]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
q1 = prepareQuery('''
  SELECT ?subclase WHERE {
    ?subclase rdfs:subClassOf ?LivingThing.
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)

for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#Animal'),)
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#Professor'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [5]:
q2 = prepareQuery('''
  SELECT DISTINCT ?persona WHERE {
    ?Class rdfs:subClassOf* ns:Person .
    ?persona rdf:type ?Class .
    }
  ''',
  initNs = { "rdfs": RDFS, "rdf": RDF, "ns" : ns}
)

for r in g.query(q2):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)
(rdflib.term.URIRef('http://somewhere#JimGonzalez'),)


**TASK 7.3: List all individuals of just "Person" or "Animal". You do not need to list the individuals of the subclasses of person (in SPARQL only)**


In [6]:
q3 = prepareQuery('''
  SELECT ?persona WHERE {
    ?subclase rdfs:subClassOf ns:LivingThing.
    ?persona rdf:type ?subclase .
    }
  ''',
  initNs = { "rdfs": RDFS, "rdf": RDF, "ns" : ns}
)

for r in g.query(q3):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#RockySmith'),)
(rdflib.term.URIRef('http://somewhere#Bugs'),)


**TASK 7.4:  List the name of the persons who know Rocky (in SPARQL only)**

In [7]:
from rdflib.namespace import FOAF

g.namespace_manager.bind('foaf', FOAF, override=False)
foaf = Namespace("http://xmlns.com/foaf/0.1/")
q4 = prepareQuery('''
    SELECT ?persona WHERE {
        ?persona foaf:knows ns:RockySmith.
    }
    ''',
    initNs={"foaf": FOAF, "ns": ns}
)

for r in g.query(q4):
  print(r)

(rdflib.term.URIRef('http://somewhere#Bugs'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**Task 7.5: List the name of those animals who know at least another animal in the graph (in SPARQL only)**

In [8]:
q5 = prepareQuery('''
    SELECT ?animal WHERE {
        ?animal rdf:type ns:Animal.
        ?animal foaf:knows ?animal2.
        ?animal2 rdf:type ns:Animal.
    }
    ''',
    initNs={"rdf": RDF, "foaf": FOAF, "ns": ns}
)

for r in g.query(q5):
  print(r)


(rdflib.term.URIRef('http://somewhere#RockySmith'),)
(rdflib.term.URIRef('http://somewhere#Bugs'),)


**Task 7.6: List the age of all living things in descending order (in SPARQL only)**

In [13]:
q6 = prepareQuery('''
    SELECT ?livingThing ?age WHERE {
        {
            ?livingThing rdf:type ns:Person.
            ?livingThing foaf:age ?age.
        }
        UNION
        {
            ?livingThing rdf:type ns:Animal.
            ?livingThing foaf:age ?age.
        }
    }
    ORDER BY DESC(?age)
   ''',
   initNs={"rdf": RDF, "foaf": FOAF, "ns": ns}
)

for r in g.query(q6):
  print(r)


(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.Literal('22', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.Literal('20', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://somewhere#RockySmith'), rdflib.term.Literal('10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://somewhere#Bugs'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
